In [12]:
#importing the utilities
from db_utilities import connect_to_database, execute_and_return_results

In [2]:
def register_user(user_id : str, username : str, email : str) -> int:
    '''
    Function to register the new user in the database
    Arguments:
        user_id   : the unique user_id created for the user
        username  : the username of the user
        email     : user's email
    Returns :
        int(0/-1) : whether the query execution was successful or not ( 0 : successful ; -1 : error)
    '''
    INSERT_USER_DETAILS = """ INSERT INTO user_details( user_id, username, email) VALUES ('{}','{}','{}');"""
    r = -1
    with connect_to_database(1) as conn :
        try:
            query = INSERT_USER_DETAILS.format(user_id,username,email)
            cursor_obj = conn.cursor()
            cursor_obj.execute(query)
            cursor_obj.close()
            #print("Query executed successfully")
            r = 0
        except Exception as e:
            #print(query)
            print("Error in executing the query for register_user : ",e)
            
    return r

In [3]:
print(register_user('Deepankar_testing','deepankar_testing','deepankar_testing1@convai.com'))

Connected to the database.
0


In [4]:
def register_api_key(user_id : str, email : str, api_key : str) -> int:
    '''
    Function to register the api key for the user in the database
    Arguments:
        user_id   : the unique user_id created for the user
        api_key   : the user's apikey
        email     : user's email
    Returns :
        int(0/-1) : whether the query execution was successful or not ( 0 : successful ; -1 : error)
    '''
    INSERT_NEW_API = """ INSERT INTO api_map( user_id, email, api_key) VALUES ('{}','{}','{}'); """
    r = -1
    with connect_to_database(1) as conn :
        try:
            query = INSERT_NEW_API.format(user_id,email, api_key)
            cursor_obj = conn.cursor()
            cursor_obj.execute(query)
            cursor_obj.close()
            #print("Query executed successfully")
            r = 0
        except Exception as e:
            #print(query)
            print("Error in executing the query for register_api_key : ",e)
            
    return r

In [5]:
print(register_api_key('Deepankar_testing','deepankar_testing1@convai.com','deepankar_api_key'))

Connected to the database.
0


In [6]:
def log_user_activity(api_key : str, service_accessed : str, source : str, user_input : str) -> int:
    '''
    Function to log users' activities in the database
    Arguments:
        api_key              : the user's apikey
        service_accessed     : which service the user accessed
        source               : from where the request originated => web gui or curl request
        user_input           : the input of the user
    Returns :
        int(0/-1)            : whether the query execution was successful or not ( 0 : successful ; -1 : error)
    '''
    INSERT_USER_ACTIVITY = """ INSERT INTO user_activity( api_key, service_accessed, source, input) VALUES ('{}','{}','{}','{}'); """
    r = -1
    with connect_to_database(1) as conn:
        try:
            query = INSERT_USER_ACTIVITY.format(api_key, service_accessed, source, user_input)
            cursor_obj = conn.cursor()
            cursor_obj.execute(query)
            cursor_obj.close()
            #print("Query executed successfully")
            r = 0
        except Exception as e:
            #print(query)
            print("Error in executing the query for log_user_activity : ",e)
            
    return r

In [7]:
print(log_user_activity('deepankar_api_key','testing','jupyter lab','unit testing'))

Connected to the database.
0


In [8]:
print(log_user_activity('deepankar_wrong_api_key','testing','jupyter lab','unit testing'))

Connected to the database.
Error in executing the query for log_user_activity :  insert or update on table "user_activity" violates foreign key constraint "user_activity_api_key_fkey"
DETAIL:  Key (api_key)=(deepankar_wrong_api_key) is not present in table "api_map".

-1


In [9]:
def get_api_key_info(email : str) -> str:
    '''
    Function to retrieve the user's api_key the database
    Arguments:
        email   : the user's email id
    Returns :
        str     : returns the latest api_key registered against the email id;
                  will return -1 in-case no api_key found in the database
    '''
    GET_API_KEY_DETAILS = """ SELECT api_key
                          FROM (SELECT api_key, generation_timestamp AS gt FROM api_map WHERE email = '{}' ORDER BY gt DESC) AS S
                          LIMIT 1; """
    api_key = "-1"
    with connect_to_database(1) as conn:
        try :
            query = GET_API_KEY_DETAILS.format(email)
            query_results = execute_and_return_results(query, conn)
        
            if len(query_results) > 0:
                api_key = str(query_results[0]['api_key'])
        except Exception as e:
            #print(query)
            print("Error in executing the query for get_api_key_info : ",e)
    
    return api_key

In [10]:
print(get_api_key_info('deepankar_testing1@convai.com'))

Connected to the database.
deepankar_api_key


In [13]:
print(get_api_key_info('deepankar_testing_wrong1@convai.com'))

Connected to the database.
-1


In [14]:
def user_login(email : str) -> dict:
    '''
    Function to retrieve the user's api_key from the database
    Arguments:
        email   : the user's email id
    Returns :
        str     : returns the latest api_key registered against the email id;
                  will return -1 in-case no api_key found in the database
    IMPNOTE : This function performs the same functionality as the get_api_key_info(). 
              Only difference is the return type.
              Havent removed it in the current build as not sure whether which function is being used in the middleman script.
              Will remove in the future iteration.
    '''
    GET_API_KEY_DETAILS = """ SELECT api_key
                          FROM (SELECT api_key, generation_timestamp AS gt FROM api_map WHERE email = '{}' ORDER BY gt DESC) AS S
                          LIMIT 1; """
    api_key = {"api_key":"-1"}
    with connect_to_database(1) as conn:
        try :
            query = GET_API_KEY_DETAILS.format(email)
            query_results = execute_and_return_results(query, conn)
        
            if len(query_results) > 0:
                api_key = query_results[0]
        except Exception as e:
            #print(query)
            print("Error in executing the query for user_login : ",e)
    
    return api_key

In [15]:
print(user_login('deepankar_testing1@convai.com'))

Connected to the database.
{'api_key': 'deepankar_api_key'}


In [17]:
print(user_login('wrong_deepankar_testing1@convai.com'))

Connected to the database.
{'api_key': '-1'}


In [18]:
def check_apiKey_existence(api_key : str) -> int:
    '''
    Function to check if the provided api_key exists in the database
    Arguments:
        api_key   : the api_key to be checked
    Returns :
        int     : returns 0 in-case the api_key is found in the database else will return -1
    '''
    r = -1 
    CHECK_API_KEY_EXISTENCE = """ SELECT * FROM api_map WHERE api_key = '{}';"""
    with connect_to_database(1) as conn:
        try :
            query = CHECK_API_KEY_EXISTENCE.format(api_key)
            query_results = execute_and_return_results(query, conn)
        
            if len(query_results) > 0:
                r = 0
        except Exception as e:
            #print(query)
            print("Error in executing the query for check_apiKey_existence : ",e)
    
    return r

In [19]:
print(check_apiKey_existence('deepankar_api_key'))

Connected to the database.
0


In [20]:
print(check_apiKey_existence('deepankar_wrong_api_key'))

Connected to the database.
-1


In [21]:
def get_all_personal_characters(user_id : str) -> list:
    '''
    Function to retrieve the characters owned by the user from the database
    Arguments:
        user_id   : the user_id to be checked against
    Returns :
        list     : returns list of dictionaries for all the characters owned by the user_id
                   if no results are found, will return empty list
    '''
    data = []
    RETRIEVE_PERSONAL_CHARACTERS = """ SELECT * FROM all_characters WHERE user_id='{}' ; """

    with connect_to_database(1) as conn:
        try :
            query = RETRIEVE_PERSONAL_CHARACTERS.format(user_id)
            query_results = execute_and_return_results(query, conn)
        
            if len(query_results) > 0:
                data = query_results
        except Exception as e:
            #print(query)
            print("Error in executing the query for get_all_personal_characters : ",e)
    return data

In [22]:
print(get_all_personal_characters('CAIPURNENDUROCKS2022AR'))

Connected to the database.
[{'character_name': 'Elon-Bit Musk', 'collection_name': 'ConvAI Curated', 'user_id': 'CAIPURNENDUROCKS2022AR', 'character_id': 'b1dd2afa-b4e4-11ec-bfda-0242ac120003', 'model_type': '2D', 'state_names': ['idle', 'talking'], 'state_links': ['https://lh3.googleusercontent.com/Tfl2fAON_DH9l-y0CTFkuWsvOFP-Z9tWPrZ-eWVaWxdbv7KNTy_XvKMieeLM6jodjAFDvuQT87aLEVR_2O_Hf8JNSpX2v18Gs-zAnFGgPJpys9CFQNyh1e0QQ1Qp-TOFAGCU8b9C=w2400', 'https://lh3.googleusercontent.com/4B1u7xYQbt_YoI1gDL-fcfuYofzDhN7idsyjOv1PX818ix7ZKeffFiF-Ti-KQdsg8Fu1-YIOOr-AbVOxk_PS2NzrmQT12IrCjWxY3ZuN4qX70sS-5u32RqXN8z4kMaegauskxUV5=w2400'], 'listing': 'public', 'voice_type': 'MALE', 'voice_pitch': 0.0, 'blockchain': None, 'contract_address': None, 'mint_address': None, 'owner_address': None, 'timestamp': Timestamp('2022-04-05 13:31:27.124306')}, {'character_name': 'Conrad', 'collection_name': None, 'user_id': 'CAIPURNENDUROCKS2022AR', 'character_id': '7c2c703c-ba98-11ec-98be-0242ac120003', 'model_type': '2D

In [23]:
print(get_all_personal_characters('CAIPURNENDUROCKS2022AR_wrong'))

Connected to the database.
[]


In [24]:
def get_character_details(char_id : str) -> dict:
    '''
    Function to retrieve the characters details from the database
    Arguments:
        char_id   : the char_id of the character
    Returns :
        dict     : returns a dict consisting of the character details
                   if no results are found, will return {"status":-1}
    '''
    data = {"status":-1}
    RETRIEVE_CHARACTER_DETAILS = """ SELECT * FROM all_characters WHERE character_id='{}' ; """

    with connect_to_database(1) as conn:
        try :
            query = RETRIEVE_CHARACTER_DETAILS.format(char_id)
            query_results = execute_and_return_results(query,conn)
        
            if len(query_results) > 0:
                data = query_results
        except Exception as e:
            #print(query)
            print("Error in executing the query for get_character_details : ",e)

    return data

In [25]:
print(get_character_details('b1dd2afa-b4e4-11ec-bfda-0242ac120003'))

Connected to the database.
[{'character_name': 'Elon-Bit Musk', 'collection_name': 'ConvAI Curated', 'user_id': 'CAIPURNENDUROCKS2022AR', 'character_id': 'b1dd2afa-b4e4-11ec-bfda-0242ac120003', 'model_type': '2D', 'state_names': ['idle', 'talking'], 'state_links': ['https://lh3.googleusercontent.com/Tfl2fAON_DH9l-y0CTFkuWsvOFP-Z9tWPrZ-eWVaWxdbv7KNTy_XvKMieeLM6jodjAFDvuQT87aLEVR_2O_Hf8JNSpX2v18Gs-zAnFGgPJpys9CFQNyh1e0QQ1Qp-TOFAGCU8b9C=w2400', 'https://lh3.googleusercontent.com/4B1u7xYQbt_YoI1gDL-fcfuYofzDhN7idsyjOv1PX818ix7ZKeffFiF-Ti-KQdsg8Fu1-YIOOr-AbVOxk_PS2NzrmQT12IrCjWxY3ZuN4qX70sS-5u32RqXN8z4kMaegauskxUV5=w2400'], 'listing': 'public', 'voice_type': 'MALE', 'voice_pitch': 0.0, 'blockchain': None, 'contract_address': None, 'mint_address': None, 'owner_address': None, 'timestamp': Timestamp('2022-04-05 13:31:27.124306')}]


In [27]:
print(get_character_details('b1dd2afa-b4e4-11ec-bfda-0242ac120503'))

Connected to the database.
{'status': -1}


In [28]:
def get_doc_store_file_link(char_id : str) -> str:
    '''
    Function to retrieve the doc store file link for a character from the database
    Arguments:
        char_id   : the char_id of the character
    Returns :
        str       : the link to the doc store file ;
                    incase no results were found, return "-1"
    '''
    RETRIEVE_GET_DOCSTORE_FILE_LINK = """ SELECT document_store_file_link FROM character_metadata WHERE character_id='{}' AND version = 0 ;"""
    doc_store_file_link = "-1"
    with connect_to_database(1) as conn:
        try:   
            query = RETRIEVE_GET_DOCSTORE_FILE_LINK.format(char_id)
            query_results = execute_and_return_results(query,conn)

            if len(query_results) > 0:
                doc_store_file_link = query_results[0]["document_store_file_link"]
        except Exception as e:
            #print(query)
            print("Error in executing the query for get_doc_store_file_link : ",e)
    return doc_store_file_link

In [29]:
print(get_doc_store_file_link('b1dd2afa-b4e4-11ec-bfda-0242ac120003'))

Connected to the database.
1649409650945467904_docStore_b1dd2afa-b4e4-11ec-bfda-0242ac120003.zip


In [30]:
print(get_doc_store_file_link('b1dd2afa-b4e4-11ec-bfda-0242ac120023'))

Connected to the database.
-1


In [31]:
def get_all_characters_from_collection(collection_name : str) -> list:
    '''
    Function to retrieve the details of all the characters from a particular collection from the database
    Arguments:
        collection_name   : the name of the collection
    Returns :
        list              : list of dictionaries consisting of character details;
                            incase no results were found, return empty list
    '''
    RETRIEVE_GET_ALL_CHARACTERS_FROM_COLLECTION = """ SELECT * FROM all_characters WHERE collection_name = '{}' ; """
    data = []
    with connect_to_database(1) as conn:
        try:   
            query = RETRIEVE_GET_ALL_CHARACTERS_FROM_COLLECTION.format(collection_name)
            query_results = execute_and_return_results(query,conn)

            if len(query_results) > 0:
                data = query_results
        except Exception as e:
            #print(query)
            print("Error in executing the query for get_all_characters_from_collection : ",e)
    return data

In [32]:
print(get_all_characters_from_collection('Cyber Beast Social Club'))

Connected to the database.
[{'character_name': 'Cyber Beast #444', 'collection_name': 'Cyber Beast Social Club', 'user_id': 'CAICONTACT2022AR', 'character_id': 'b23b4324-b4e4-11ec-bfda-0242ac120003', 'model_type': '2D', 'state_names': ['idle', 'talking'], 'state_links': ['https://lh3.googleusercontent.com/xuARcuqCY9UsSF3jUzMpgzAxvHIYcpDXMmNgsNv1U6U4EMVgNAIbAcLmWbict4p4rbiH_DNDvYwsxm-CbLEOCCp9lZhKj18yiQbSiSRmJBCRe0_RRrTzv6BGuz7fOtWY-M1_rfMd=w2400', 'https://lh3.googleusercontent.com/w7FR_OgaguSyhPueURUspAbMvUjoZJe3YGhAsHbFjRNobmz4Njh9ecwwo1ggBfNtce8TvtEnPNqJH50lh6TK41kG1igQqv5A7xCvXk5FPcNK-Y2GQkFrCGHKrtmI0LtmlGAKE9Rc=w2400'], 'listing': 'public', 'voice_type': 'MALE', 'voice_pitch': 0.0, 'blockchain': None, 'contract_address': None, 'mint_address': None, 'owner_address': None, 'timestamp': Timestamp('2022-04-05 13:31:27.741241')}]


In [33]:
print(get_all_characters_from_collection('Cyber Beast Social Club01'))

Connected to the database.
[]


In [34]:
def get_all_character_collections() -> list:
    '''
    Function to retrieve the details of all the collections from the database
    Arguments: None
    Returns :
        list              : list of dictionaries consisting of character details;
                            incase no results were found, return empty list
    '''
    RETRIEVE_COLLECTION_STATS = """ SELECT * FROM character_collections; """
    data = []
    with connect_to_database(1) as conn:
        try:   
            query = RETRIEVE_COLLECTION_STATS
            query_results = execute_and_return_results(query,conn)

            if len(query_results) > 0:
                data = query_results
        except Exception as e:
            #print(query)
            print("Error in executing the query for get_all_character_collections : ",e)
    return data

In [35]:
print(get_all_character_collections())

Connected to the database.
[{'collection_id': '20ebe320-b4e3-11ec-bfda-0242ac120003', 'collection_name': 'Cyber Beast Social Club', 'collection_logo': 'https://lh3.googleusercontent.com/B1F3Isy-5ZOirXDwvEItuQBWrApHEV_xPb0QyJqPJ6bIskEHxLzknicMJSLk65yyTNwVrAv5IwhL5slMeJ6oQrY9px0qq14FjKUmm7eiW91dnATZ_uiE7vmgag9qbPc_jm__WpNi=w2400', 'collection_banner': None, 'collection_website': None, 'collection_owner_id': 'CAICONTACT2022AR', 'collection_description': None, 'timestamp': Timestamp('2022-04-05 13:20:14.453781')}, {'collection_id': '211267fc-b4e3-11ec-bfda-0242ac120003', 'collection_name': 'ConvAI Curated', 'collection_logo': None, 'collection_banner': 'https://convai.com', 'collection_website': None, 'collection_owner_id': 'CAIPURNENDUROCKS2022AR', 'collection_description': None, 'timestamp': Timestamp('2022-04-05 13:20:14.707163')}, {'collection_id': '21436b90-b4e3-11ec-bfda-0242ac120003', 'collection_name': 'OKACTP', 'collection_logo': 'https://lh3.googleusercontent.com/6Cx-EBPWPM9cctfS

In [36]:
def get_username(user_id : str) -> str:
    '''
    Function to retrieve the username for a user id from the database
    Arguments:
        user_id   : user_id whose username needs to be retrieved
    Returns :
        str       : the username for the provided user_id;
                    incase no results were found, return "-1"
    '''
    username = "-1"
    RETRIEVE_USERNAME = """ SELECT username FROM user_details WHERE user_id='{}' ;"""
    with connect_to_database(1) as conn:
        try:   
            query = RETRIEVE_USERNAME.format(user_id)
            query_results = execute_and_return_results(query,conn)

            if len(query_results) > 0:
                username = query_results[0]["username"]
        except Exception as e:
            #print(query)
            print("Error in executing the query for get_username : ",e)
    return username

In [38]:
print(get_username('Deepankar_testing'))

Connected to the database.
deepankar_testing


In [39]:
print(get_username('Deepankar_testing_wrong'))

Connected to the database.
-1


In [45]:
def get_user_count() -> int:
    '''
    Function to retrieve the total number of registered user in the database
    Arguments : None
    Returns   :
        int       : total number of registered users
    '''
    usercount = -1
    RETRIEVE_USER_COUNT = """ SELECT COUNT(user_id) AS usercount FROM user_details  ;"""
    with connect_to_database(1) as conn:
        try:   
            query = RETRIEVE_USER_COUNT
            query_results = execute_and_return_results(query,conn)

            if len(query_results) > 0:
                usercount = int(query_results[0]["usercount"])
        except Exception as e:
            #print(query)
            print("Error in executing the query for get_user_count : ",e)
    return usercount

In [46]:
print(get_user_count())

Connected to the database.
54


In [53]:
def insert_new_character(
    character_name : str,
    user_id : str,
    model_type : str,
    state_names : list,
    state_links : list,
    listing : str,
    voice_type : str,
    voice_pitch : float,
    blockchain : str,
    contract_address : str,
    mint_address : str,
    owner_address : str,
    collection_name : str = 'convai_default_collection'
) -> str:
    '''
    Function to insert a new character into the database
    Arguments:
        character_name  : the name of the character
        user_id         : the user id of the owner
        model_type      : whether 2D or 3D
        collection_name : name of the collection, by default it will be a part of 'convai_default_collection'
        state_names     : available states for the character 
        state_links     : links to the available state
        listing         : public or private
        voice_type      : type of voice for the character
        voice_pitch     : pitch value for the character
        blockchain      : <will ask Himadri da to fill in>
        contract_address: <will ask Himadri da to fill in>
        mint_address    : <will ask Himadri da to fill in>
        owner_address   : <will ask Himadri da to fill in>
    Returns :
        str       : the character id for the new character;
                    incase no results were found, return "-1"
    '''

    char_id = "-1"
    state_names = "{" + ",".join(state_names) + "}"
    state_links = "{" + ",".join(state_links) + "}"

    INSERT_CHARACTER_INTO_ALLCHARACTERS = """ INSERT INTO all_characters 
                                              (character_name, collection_name, user_id, model_type, state_names, state_links, listing, voice_type, voice_pitch, blockchain, contract_address, mint_address, owner_address) 
                                              VALUES
                                              ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}'); """
    RETRIEVE_CHARACTERID_FROM_ALLCHARACTERS = """SELECT character_id FROM all_characters 
                                                 WHERE character_name = '{}' AND user_id = '{}' AND collection_name = '{}' 
                                                 ORDER BY timestamp DESC; """
    

    with connect_to_database(1) as conn:
        try:   
            query01 = INSERT_CHARACTER_INTO_ALLCHARACTERS.format(character_name, collection_name, user_id, model_type, state_names, state_links, listing, voice_type, voice_pitch, blockchain, contract_address, mint_address, owner_address)
            cursor_obj = conn.cursor()
            cursor_obj.execute(query01)
            cursor_obj.close()

            query02 = RETRIEVE_CHARACTERID_FROM_ALLCHARACTERS.format(character_name, user_id, collection_name)
            query_results = execute_and_return_results(query02,conn)

            if len(query_results) > 0:
                char_id = query_results[0]["character_id"]
        except Exception as e:
            #print(query)
            print("Error in executing the query for insert_new_character : ",e)
        
    return char_id

In [55]:
print(insert_new_character(
    character_name = 'Tony Stark duplicate',
    user_id = 'Deepankar_testing',
    model_type = '3D',
    state_names = ['state01','state02','state03'],
    state_links = ['link01','link02','link03'],
    listing = 'Public',
    voice_type = 'MALE',
    voice_pitch = 0.0,
    blockchain = 'NULL',
    contract_address = 'NULL',
    mint_address = 'NULL',
    owner_address = 'NULL',
))

Connected to the database.
eb0d9ca2-bcae-11ec-9b34-0242ac120003
